# Vitessce Widget Tutorial

# Visualization of Space Ranger outputs for 10x Visium


In [94]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    OmeTiffWrapper,
    MultiImageWrapper,
    AnnDataWrapper,
    CoordinationLevel as CL,
)
from vitessce.data_utils import (
   rgb_img_to_ome_tiff,
    VAR_CHUNK_SIZE
)
import os
from os.path import join
import requests
from anndata import AnnData, read_hdf
import tifffile
import tarfile
import scanpy as sc
import pandas as pd
import imageio.v3 as iio
import json

## 0. Download data

https://www.10xgenomics.com/resources/datasets/gene-protein-expression-library-of-human-tonsil-cytassist-ffpe-2-standard

In [48]:
DATA_DIR = join("data", "human_tonsil")
PROCESSED_DIR = join(DATA_DIR, "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)
spatial_filepath = join(DATA_DIR, "spatial.tar.gz")
matrix_filepath = join(DATA_DIR, "matrix.h5")
analysis_filepath = join(DATA_DIR, "analysis.tar.gz")

base_url = 'https://cf.10xgenomics.com/samples/spatial-exp/2.1.0/CytAssist_FFPE_Protein_Expression_Human_Tonsil/CytAssist_FFPE_Protein_Expression_Human_Tonsil'

# Download
r_spatial = requests.get(f'{base_url}_spatial.tar.gz')
with open(spatial_filepath, 'wb') as f:
    f.write(r_spatial.content)

r_matrix = requests.get(f'{base_url}_filtered_feature_bc_matrix.h5')
with open(matrix_filepath, 'wb') as f:
    f.write(r_matrix.content)

r_analysis = requests.get(f'{base_url}_analysis.tar.gz')
with open(analysis_filepath, 'wb') as f:
    f.write(r_analysis.content)
    
# Un-tar
with tarfile.open(spatial_filepath, "r:gz") as tar:
    tar.extractall(path = DATA_DIR)
with tarfile.open(analysis_filepath, "r:gz") as tar:
    tar.extractall(path = DATA_DIR)

## 1. Convert data to Vitessce-compatible formats

We need to convert the proprietary 10x Genomics formats to open formats that are compatible with Vitessce such as OME-TIFF and AnnData

In [70]:
adata = sc.read_10x_h5(matrix_filepath)

/Users/mkeller/software/miniconda3/envs/vitessce-python-notebooks/lib/python3.9/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/mkeller/software/miniconda3/envs/vitessce-python-notebooks/lib/python3.9/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [71]:
tsne_df = pd.read_csv(join(DATA_DIR, "analysis", "tsne", "gene_expression_2_components", "projection.csv"), index_col=0)

In [72]:
adata.obsm['X_tsne'] = tsne_df.loc[adata.obs.index.tolist()].values

In [73]:
with open(join(DATA_DIR, "spatial", "scalefactors_json.json"), "r") as f:
    scale_factors = json.load(f)

In [74]:
scale_factors

{'regist_target_img_scalef': 0.1533272,
 'tissue_hires_scalef': 0.051109068,
 'tissue_lowres_scalef': 0.01533272,
 'fiducial_diameter_fullres': 383.3196076845379,
 'spot_diameter_fullres': 255.54640512302527}

In [75]:
pos_df = pd.read_csv(join(DATA_DIR, "spatial", "tissue_positions.csv"), index_col=0)[['pxl_row_in_fullres', 'pxl_col_in_fullres']].loc[adata.obs.index.tolist()]

In [76]:
adata.obsm['X_spatial'] = pos_df.values * scale_factors['tissue_hires_scalef']

In [80]:
adata.write_zarr(join(PROCESSED_DIR, "human_tonsil.h5ad.zarr"), chunks=(adata.shape[0], VAR_CHUNK_SIZE))

In [81]:
img_arr = iio.imread(join(DATA_DIR, "spatial", "tissue_hires_image.png"))
img_arr = img_arr.transpose((2, 0, 1))
img_arr.shape

(3, 1634, 2000)

In [82]:
rgb_img_to_ome_tiff(img_arr, join(PROCESSED_DIR, "human_tonsil.ome.tiff"), img_name="Image", axes="CYX")

## 1. Configure Vitessce


In [92]:
vc = VitessceConfig(schema_version="1.0.16", name='Tonsil')
dataset = vc.add_dataset(name='Space Ranger outputs').add_object(AnnDataWrapper(
    adata_path=join(PROCESSED_DIR, "human_tonsil.h5ad.zarr"),
    obs_feature_matrix_path="X",
    obs_embedding_paths=["obsm/X_tsne"],
    obs_embedding_names=["t-SNE"],
    obs_locations_path="obsm/X_spatial",
    coordination_values={
        "obsType": "spot"
    }
)).add_object(OmeTiffWrapper(
    img_path=join(PROCESSED_DIR, "human_tonsil.ome.tiff")
))

spatial = vc.add_view("spatial", dataset=dataset)
lc = vc.add_view("layerController", dataset=dataset)
heatmap = vc.add_view("heatmap", dataset=dataset)
scatterplot = vc.add_view("scatterplot", dataset=dataset, mapping="t-SNE")

vc.link_views([spatial, lc, heatmap, scatterplot], ["obsType"], ["spot"])

vc.layout((spatial | lc) / (heatmap | scatterplot));

## 2. Create the Vitessce widget

In [93]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.16', 'name': 'Tonsil', 'description': '', 'datasets': [{'uid': 'A', 'na…